## Лабораторная работа 5

In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col}
import org.apache.spark.ml.classification.{RandomForestClassifier, GBTClassifier}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.ml.feature.{CountVectorizer, StringIndexer, VectorAssembler, StandardScaler, MinMaxScaler}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator, CrossValidatorModel}
import org.apache.spark.sql.DataFrame

In [2]:
val spark = SparkSession.builder.appName("Spark ML").getOrCreate()

spark = org.apache.spark.sql.SparkSession@1b99fba4


org.apache.spark.sql.SparkSession@1b99fba4

In [8]:
spark

org.apache.spark.sql.SparkSession@1b99fba4

# Data preparation

In [11]:
var trainDf = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("slaba05data/lab05_train.csv")
    //.drop("_c0", "ID")

trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 _c0                         | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M       

trainDf = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [12]:
val categoriesCols = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)

categoriesCols = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

In [13]:
trainDf = trainDf.drop(categoriesCols:_*).na.fill(0)

trainDf = [_c0: int, ID: int ... 102 more fields]


[_c0: int, ID: int ... 102 more fields]

In [14]:
val Array(trainDfTrain, trainDfTest) = trainDf.randomSplit(Array(0.8, 0.2))

trainDfTrain = [_c0: int, ID: int ... 102 more fields]
trainDfTest = [_c0: int, ID: int ... 102 more fields]


[_c0: int, ID: int ... 102 more fields]

# Train model

In [15]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.feature.{VectorAssembler}

In [16]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(trainDfTrain.drop("TARGET").columns)
    .setOutputCol("features")

vectorAssembler = vecAssembler_ec71d27b3324


vecAssembler_ec71d27b3324

In [17]:
val trainDfCleaned = vectorAssembler.transform(trainDfTrain).cache

trainDfCleaned = [_c0: int, ID: int ... 103 more fields]


[_c0: int, ID: int ... 103 more fields]

In [18]:
trainDfCleaned.show(1, 0, true)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 _c0                        | 0                                                                                                                                                                                                                                                                                                                                    
 ID                         | 146841                                                                                                                                                                                                                                                            

 CR_PROD_CNT_TOVR           | 0                                                                                                                                                                                                                                                                                                                                    
 CR_PROD_CNT_PIL            | 0                                                                                                                                                                                                                                                                                                                                    
 SUM_TRAN_CLO_TENDENCY1M    | 0.0                                                                                                                                                                                                                                                               

In [20]:
// Train GBTClassifier
val randomForestModel = new GBTClassifier()
  .setLabelCol("TARGET")
  .setFeaturesCol("features")
  .setFeatureSubsetStrategy("auto").fit(trainDfCleaned)

randomForestModel = GBTClassificationModel (uid=gbtc_9a2a45716a37) with 20 trees


GBTClassificationModel (uid=gbtc_9a2a45716a37) with 20 trees

In [21]:
val predictions = randomForestModel.transform(
    vectorAssembler.transform(trainDfTest)
).cache

predictions
    .select(
        "TARGET",
        "features",
        "rawPrediction",
        "probability",
        "prediction"
    )
    .show(5)

+------+--------------------+--------------------+--------------------+----------+
|TARGET|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|(103,[0,1,3,19,21...|[0.64699126287020...|[0.78482051014152...|       0.0|
|     0|(103,[0,1,5,11,13...|[1.49247997354918...|[0.95189002385542...|       0.0|
|     0|(103,[0,1,3,9,10,...|[1.49247997354918...|[0.95189002385542...|       0.0|
|     1|(103,[0,1,21,31,3...|[0.91964075846602...|[0.86286371153872...|       0.0|
|     0|(103,[0,1,3,9,10,...|[1.22115493115346...|[0.91999726401990...|       0.0|
+------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



predictions = [_c0: int, ID: int ... 106 more fields]


[_c0: int, ID: int ... 106 more fields]

In [22]:
predictions
    .select(
        "probability"
    )
    .show(1, 0, true)

-RECORD 0-----------------------------------------------
 probability | [0.7848205101415283,0.21517948985847168] 
only showing top 1 row



In [23]:
predictions.select("probability").printSchema()

root
 |-- probability: vector (nullable = true)



In [24]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [25]:
predictions
    .select(vectorToArrayUdf(col("probability")).getItem(1).alias("prediction_probability"))
    .show(5)

+----------------------+
|prediction_probability|
+----------------------+
|   0.21517948985847168|
|   0.04810997614457979|
|   0.04810997614457979|
|   0.13713628846127657|
|   0.08000273598009933|
+----------------------+
only showing top 5 rows



In [26]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [27]:
val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("TARGET")
    .setRawPredictionCol("prediction")
    .setMetricName("areaUnderROC")

val accuracy = evaluator
    .evaluate(
        predictions.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    )

evaluator = binEval_a3bb4b576c74
accuracy = 0.816889615437109


0.816889615437109

# Predict

In [29]:
val testDfData = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("slaba05data/lab05_test.csv")
    .na.fill(0)

testDfData = [_c0: int, ID: int ... 114 more fields]


lastException: Throwable = null


[_c0: int, ID: int ... 114 more fields]

In [30]:
val realPredictions = randomForestModel.transform(vectorAssembler.transform(testDfData)).cache

realPredictions = [_c0: int, ID: int ... 118 more fields]


[_c0: int, ID: int ... 118 more fields]

In [31]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .show(5)

+------+--------------------+
|    ID|          prediction|
+------+--------------------+
|519130| 0.15198967884329895|
|234045|0.050341442227190614|
|401256| 0.04984725172594551|
|551070| 0.06576217411631813|
|367285| 0.04979342194150793|
+------+--------------------+
only showing top 5 rows



In [38]:
realPredictions

lastException: Throwable = null


[_c0: int, ID: int ... 118 more fields]

In [2]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .withColumnRenamed("prediction", "TARGET")
    .write
    .mode("overwrite")
    .format("csv")
    .save("/user/alexander.andreevskiy/labs/lab05/lab05.csv")

Name: Syntax Error.
Message: 
StackTrace: 

In [ ]:
/user/alexander.andreevskiy/labs/lab05/data/lab05_test.csv

In [36]:
import sys.process._

"pwd" !

/data/home/alexander.andreevskiy


0

In [31]:
spark.stop()